In [ ]:
# import os
# import json
# import random
# import shutil

# # Define dataset directory
# DATASET_DIR = r"/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/Json"  # Change this if your dataset is in a different folder

# # Define output directories for each split
# OUTPUT_DIRS = {
#     "train": r"/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/Train_Folder",
#     "valid": r"/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/Validation_Folder",
#     "test": r"/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/Test_Folder"
# }

# # Create separate folders for each split
# for split, output_dir in OUTPUT_DIRS.items():
#     os.makedirs(output_dir, exist_ok=True)

# # Get all JSON files
# json_files = [f for f in os.listdir(DATASET_DIR) if f.endswith(".json")]

# # Shuffle dataset
# random.shuffle(json_files)

# # Define split sizes
# total_files = len(json_files)
# train_size = int(0.8 * total_files)
# valid_size = int(0.1 * total_files)

# # Split files
# train_files = json_files[:train_size]
# valid_files = json_files[train_size:train_size + valid_size]
# test_files = json_files[train_size + valid_size:]

# # Move files to respective folders
# for split, files in [("train", train_files), ("valid", valid_files), ("test", test_files)]:
#     for file in files:
#         src_path = os.path.join(DATASET_DIR, file)
#         dest_path = os.path.join(OUTPUT_DIRS[split], file)
#         shutil.move(src_path, dest_path)

# print("Files successfully moved into separate folders: train, valid, and test.")


In [ ]:
# Making ALL json file

import os
import json

# Define base dataset directory
DATASET_DIR = r"/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File"  # Update this if your dataset folder is named differently

# Define dataset structure
data_splits = {"train": [], "valid": [], "test": []}

# Loop through train, valid, and test folders
# Changed the loop to use the correct keys from data_splits
for split_folder, split_key in [("Train_Folder", "train"), ("Validation_Folder", "valid"), ("Test_Folder", "test")]:
    folder_path = os.path.join(DATASET_DIR, split_folder)

    # Get all JSON files in the split folder
    json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

    # Read and append data
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        with open(file_path, "r", encoding="utf-8") as f:
            file_data = json.load(f)
            # Append to the correct list in data_splits using split_key
            data_splits[split_key].append(file_data)

# Save the combined dataset as dataset.json
output_path = os.path.join(DATASET_DIR, "dataset.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data_splits, f, indent=4)

print(f"Combined dataset saved at: {output_path}")

Combined dataset saved at: /content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/dataset.json


In [ ]:
#Fixing ALl josn syntax

import json

input_file = '/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/dataset.json'  # Replace with your input JSON file
output_file = '/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/FixedJson.json'  # Replace with the desired output JSON file

# Read the input JSON file
with open(input_file, 'r') as f:
    data = json.load(f)

# Convert the structure
for key in data:
    new_list = []
    for sublist in data[key]:
        new_list.extend(sublist)
    data[key] = new_list

# Write the updated structure to the output JSON file
with open(output_file, 'w') as f:
    json.dump(data, f, indent=4)

print(f"Converted structure saved to {output_file}")


Converted structure saved to /content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/FixedJson.json


In [ ]:
import json
from transformers import AutoTokenizer

# Path to dataset
dataset_path = r"/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/All_Json_File/FixedJson.json"

# Load dataset
with open(dataset_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Load CodeT5-220M tokenizer
model_name = "Salesforce/codet5-large"  # or "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure tokenizer has a pad token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize function
def tokenize_dataset(split, dataset, tokenizer):
    print(f"Processing {split} dataset:")
    tokenized_data = []

    for i, entry in enumerate(dataset.get(split, [])):
        source_code = entry.get('source', "")
        target_code = entry.get('target', "")

        # Tokenize source and target code
        source_tokens = tokenizer(source_code, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        target_tokens = tokenizer(target_code, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

        # Convert tensors to lists
        tokenized_data.append({
            'source_input_ids': source_tokens['input_ids'].squeeze().tolist(),
            'source_attention_mask': source_tokens['attention_mask'].squeeze().tolist(),
            'target_input_ids': target_tokens['input_ids'].squeeze().tolist(),
            'target_attention_mask': target_tokens['attention_mask'].squeeze().tolist()
        })

        if (i + 1) % 10 == 0:
            print(f"Processed {i+1} entries in {split} dataset.")

    # Save tokenized data
    with open(f"tokenized_{split}.json", "w", encoding="utf-8") as f_out:
        json.dump(tokenized_data, f_out, ensure_ascii=False, indent=4)

    print(f"Finished processing {split} dataset.")

# Tokenize train, valid, and test datasets
for split in ['train', 'valid', 'test']:
    tokenize_dataset(split, data, tokenizer)

print("Tokenization complete! 🚀 Data saved successfully.")


Processing train dataset:
Processed 10 entries in train dataset.
Processed 20 entries in train dataset.
Processed 30 entries in train dataset.
Processed 40 entries in train dataset.
Processed 50 entries in train dataset.
Processed 60 entries in train dataset.
Processed 70 entries in train dataset.
Processed 80 entries in train dataset.
Processed 90 entries in train dataset.
Processed 100 entries in train dataset.
Processed 110 entries in train dataset.
Processed 120 entries in train dataset.
Processed 130 entries in train dataset.
Processed 140 entries in train dataset.
Processed 150 entries in train dataset.
Processed 160 entries in train dataset.
Processed 170 entries in train dataset.
Processed 180 entries in train dataset.
Processed 190 entries in train dataset.
Processed 200 entries in train dataset.
Processed 210 entries in train dataset.
Processed 220 entries in train dataset.
Processed 230 entries in train dataset.
Processed 240 entries in train dataset.
Processed 250 entries i

In [ ]:
## json files into a pytorch dataset
import json
import torch
from torch.utils.data import Dataset

class CodeDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": torch.tensor(item["source_input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(item["source_attention_mask"], dtype=torch.long),
            "labels": torch.tensor(item["target_input_ids"], dtype=torch.long),
        }

# Load datasets
train_dataset = CodeDataset("/content/tokenized_train.json")
valid_dataset = CodeDataset("/content/tokenized_valid.json")
test_dataset = CodeDataset("/content/tokenized_test.json")

print(f"Train samples: {len(train_dataset)}, Valid samples: {len(valid_dataset)}, Test samples: {len(test_dataset)}")


Train samples: 387, Valid samples: 10, Test samples: 15


In [ ]:
!pip install datasets


In [ ]:
from datasets import Dataset

def convert_to_hf_dataset(custom_dataset):
    return Dataset.from_list([
        {
            "input_ids": ex["input_ids"],
            "attention_mask": ex["attention_mask"],
            "labels": ex["labels"]
        } for ex in custom_dataset
    ])

# Convert train, validation, and test datasets
train_dataset = convert_to_hf_dataset(train_dataset)
valid_dataset = convert_to_hf_dataset(valid_dataset)
test_dataset = convert_to_hf_dataset(test_dataset)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "Salesforce/CodeT5p-220m"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Correct model type
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_name)





In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load the correct model for code translation
model_name = "Salesforce/CodeT5p-220m"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available, handling potential memory issues
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
try:
    model.to(device)
    print("✅ Model loaded and moved to:", device)
except RuntimeError as e:
    if "out of memory" in str(e):
        print("❌ CUDA out of memory. Trying to reduce model size or batch size...")
        # Option 1: Clear GPU cache
        torch.cuda.empty_cache()
        # Option 2: Reduce batch size (if applicable)
        # Option 3: If still out of memory, consider a smaller model or gradient accumulation
        try:
            model.to(device)  # Retry after cache clearing
            print("✅ Model loaded and moved to:", device)  # If successful after retry
        except RuntimeError as e:
            print(f"❌ Still out of memory: {e}")
    else:
        raise e  # Re-raise other exceptions

✅ Model loaded and moved to: cuda


In [ ]:
from transformers import Trainer, TrainingArguments
import torch

# Define batch size (lower to avoid memory issues)
BATCH_SIZE = 2

training_args = TrainingArguments(
    output_dir="./results",  # Directory to save checkpoints
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save checkpoint at each epoch
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_dir="./logs",
    logging_steps=100,  # Log every 200 steps
    save_total_limit=None,  # Keep all checkpoints (change if storage is limited)
    num_train_epochs=3,  # More epochs for gradual training
    learning_rate=5e-5,
    weight_decay=0.01,
    push_to_hub=False,  # Set to True if using Hugging Face Hub
    fp16=True if torch.cuda.is_available() else False,  # Mixed precision for GPUs
    load_best_model_at_end=True,  # Resume training with the best checkpoint
    resume_from_checkpoint=True  # Auto-resume training
)

print("✅ Training arguments set with checkpoint saving enabled!")


✅ Training arguments set with checkpoint saving enabled!


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
## checking dataset are ready or not
# Check if the datasets exist
if "train_dataset" not in globals() or "valid_dataset" not in globals() or "test_dataset" not in globals():
    raise ValueError("❌ `train_dataset`, `valid_dataset`, or `test_dataset` is not defined. Load your datasets first.")

print("✅ Datasets are ready!")


✅ Datasets are ready!


In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq

# Define the data collator (for padding sequences properly)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("✅ Trainer initialized!")


✅ Trainer initialized!


<ipython-input-62-96f25048e8fd>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()
# Save the model and tokenizer
trainer.save_model("/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/Add_Comments")
tokenizer.save_pretrained("/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/Add_Comments")


NameError: name 'trainer' is not defined

In [37]:
## Loading the Fine tuning model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define model path (update this if needed)
model_path = "/content/drive/MyDrive/C_TO_PY--On My dataset/Dataset_Creation/Add_Comments"  # Path where you saved the model

# Load the fine-tuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

print("✅ Model and tokenizer loaded successfully!")


✅ Model and tokenizer loaded successfully!


In [38]:
import torch

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate_c_to_python(c_code):
    """
    Function to translate C code to Python using the fine-tuned CodeT5 model.
    """
    # Tokenize input
    inputs = tokenizer(c_code, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Generate translation
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=512)

    # Decode output
    translated_code = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return translated_code


In [74]:
# Sample C program (you can replace this with any C code)
c_code_example = """
#include <stdio.h>

// Function to calculate the square of a number
int square(int num) {
    return num * num;
}

int main() {
    int number;

    // Prompt the user for input
    printf("Enter a number: ");
    scanf("%d", &number);

    // Call the function and display the result
    printf("The square of %d is %d.\n", number, square(number));

    return 0;
}


"""

# Generate Python translation
translated_python_code = translate_c_to_python(c_code_example)

print("\n🔹 **Generated Python Code:**\n")
print(translated_python_code)



🔹 **Generated Python Code:**

# Function to calculate the square of a number
def square(num):
    return num ** 2


In [ ]:
import pickle
import torch

# Move the model to CPU before saving
model.to(torch.device("cpu"))

# Create a dictionary to store the model and tokenizer
deployment_data = {
    "model": model,
    "tokenizer": tokenizer
}

# Save the dictionary to a pickle file
with open("Model.pkl", "wb") as f:
    pickle.dump(deployment_data, f)

print("✅ Model and tokenizer saved to Model.pkl on CPU!")

✅ Model and tokenizer saved to Model.pkl on CPU!
